In [1]:
from data_cleaning import DataCleaning
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
import pandas as pd
import re
import numpy as np
import tabula
import os

In [47]:
config_dir = "/Users/imobassey/Desktop/DevOps/Git/multinational-retail-data-centralisation299/"  
db_connector = DatabaseConnector(config_dir)
data_extract = DataExtractor()
data_cleaner = DataCleaning()
aws_keys = db_connector.aws_credentials()
s3_address = 's3://data-handling-public/products.csv'
products_df = data_extract.extract_from_s3(s3_address, aws_keys['aws_access_key_id'], aws_keys['aws_secret_access_key'])
print(products_df)
# products_df = data_cleaner.convert_product_weights(products_df)
# clean_products_df = data_cleaner.clean_products_data(products_df)
# print(clean_products_df)

      Unnamed: 0                                 product_name product_price  \
0              0  FurReal Dazzlin' Dimples My Playful Dolphin        £39.99   
1              1          Tiffany's World Day Out At The Park        £12.99   
2              2          Tiffany's World Pups Picnic Playset         £7.00   
3              3     Tiffany's World Wildlife Park Adventures        £12.99   
4              4                      Cosatto Cosy Dolls Pram        £30.00   
...          ...                                          ...           ...   
1848        1848    Goodmans Qi Autosense In Car Phone Holder        £15.00   
1849        1849    Goodmans Qi Wireless Charging Phone Stand        £15.00   
1850        1850                     RAC 12V Wet & Dry Vacuum        £18.00   
1851        1851    Goodmans Qi Autosense In Car Phone Holder        £15.00   
1852        1852               Beldray Telescopic Ladder 2.9m        £69.00   

      weight        category            EAN  date_a

In [51]:
products_df.columns

Index(['Unnamed: 0', 'product_name', 'product_price', 'weight', 'category',
       'EAN', 'date_added', 'uuid', 'removed', 'product_code'],
      dtype='object')

In [52]:
# Drop the Unnamed: 0 column
products_df.drop(columns=['Unnamed: 0'], inplace=True)
# Remove whitespace from column names and values
products_df.columns = products_df.columns.str.strip().str.lower()
# Standardize 'removed' column values
products_df['removed'] = products_df['removed'].str.strip().str.replace('Still_avaliable', 'Still_available')
# Remove non-numeric values from product_price
products_df['product_price'] = products_df['product_price'].replace('[£]', '', regex=True)
products_df = products_df[pd.to_numeric(products_df['product_price'], errors='coerce').notna()]
products_df['product_price'] = products_df['product_price'].astype(float)
# Convert date_added to datetime
products_df['date_added'] = pd.to_datetime(products_df['date_added'], errors='coerce').dt.date
products_df = products_df.dropna(subset=['date_added'])





/var/folders/yq/r9wcdltx4655gh042jpyr47c0000gn/T/ipykernel_46881/1317953698.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_df['product_price'] = products_df['product_price'].astype(float)
/var/folders/yq/r9wcdltx4655gh042jpyr47c0000gn/T/ipykernel_46881/1317953698.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_df['date_added'] = pd.to_datetime(products_df['date_added'], errors='coerce').dt.date


In [53]:
var_1 = list(products_df['product_price'])
var_1

[39.99,
 12.99,
 7.0,
 12.99,
 30.0,
 12.99,
 7.0,
 12.99,
 12.99,
 89.99,
 24.99,
 24.99,
 24.99,
 12.99,
 12.99,
 22.0,
 22.0,
 20.0,
 16.99,
 12.99,
 39.99,
 12.99,
 12.99,
 12.99,
 25.0,
 45.0,
 24.99,
 12.99,
 18.99,
 12.99,
 29.99,
 12.99,
 12.99,
 15.99,
 30.0,
 12.99,
 12.99,
 19.99,
 12.99,
 12.99,
 12.99,
 25.0,
 45.0,
 24.99,
 12.99,
 18.99,
 12.99,
 29.99,
 12.99,
 12.99,
 15.99,
 30.0,
 12.99,
 12.99,
 19.99,
 11.99,
 14.99,
 32.99,
 20.0,
 120.0,
 10.0,
 21.99,
 30.0,
 12.99,
 12.99,
 12.99,
 29.99,
 12.99,
 7.0,
 12.99,
 12.99,
 12.99,
 24.99,
 13.99,
 11.99,
 29.99,
 16.99,
 16.99,
 32.99,
 19.99,
 16.99,
 30.99,
 12.99,
 270.0,
 12.99,
 40.99,
 42.99,
 71.99,
 34.99,
 52.99,
 13.0,
 12.99,
 2.25,
 3.0,
 3.5,
 4.0,
 4.0,
 4.0,
 12.99,
 12.99,
 12.99,
 24.99,
 26.99,
 19.99,
 19.99,
 17.99,
 32.99,
 19.99,
 7.0,
 20.0,
 12.99,
 12.99,
 25.0,
 11.99,
 12.99,
 14.99,
 14.99,
 13.99,
 12.99,
 24.99,
 26.99,
 19.99,
 19.99,
 17.99,
 32.99,
 19.99,
 7.0,
 20.0,
 12.99,
 12.99

In [54]:
products_df

,product_name,product_price,weight,category,ean,date_added,uuid,removed,product_code
0,FurReal Dazzlin' Dimples My Playful Dolphin,39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_available,R7-3126933h
1,Tiffany's World Day Out At The Park,12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_available,C2-7287916l
2,Tiffany's World Pups Picnic Playset,7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_available,S7-1175877v
3,Tiffany's World Wildlife Park Adventures,12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,Cosatto Cosy Dolls Pram,30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_available,B6-2596063a
...,...,...,...,...,...,...,...,...,...
1848,Goodmans Qi Autosense In Car Phone Holder,15.00,125g,diy,8185827751600,2020-08-28,3ff21cb1-8e90-499d-b5a0-6839fb83e70f,Still_available,x4-2227164k
1849,Goodmans Qi Wireless Charging Phone Stand,15.00,134g,diy,9769724533935,2012-05-20,a7910f74-c2b6-4144-98b7-30ca68f34d2d,Still_available,i3-8338545Z
1850,RAC 12V Wet & Dry Vacuum,18.00,820g,diy,3113855860327,1993-11-28,66d3efa0-f480-4da9-893b-e8aaff53e27b,Still_available,n4-1642658b
1851,Goodmans Qi Autosense In Car Phone Holder,15.00,125g,diy,9195087467400,2016-09-20,efb472f1-8f00-448e-a874-9fa9812eeab5,Still_available,A3-7619070S
